In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/ratings.csv',index_col=0)
df=df[['장소','아이디','평점']]
df

,장소,아이디,평점
0,양재시민의숲,Jong soon Jang,4
1,양재시민의숲,전찬우,5
2,양재시민의숲,강종수,5
3,양재시민의숲,김홍일,5
4,양재시민의숲,최민선,4
...,...,...,...
26691,N서울타워,Yorick de Wid,4
26692,N서울타워,최승영,4
26693,N서울타워,c.hooney Kim,5
26694,N서울타워,Sophie Nguyen,5


In [3]:
a = df['아이디'].value_counts() >= 5
a.sum()

32260

In [4]:
def preprocessing(df, n):
    min_id = df['아이디'].value_counts() >= n
    min_id = min_id[min_id].index.to_list()
    df = df[df['아이디'].isin(min_id)]

    min_place = df['장소'].value_counts() >= n
    min_place = min_place[min_place].index.to_list()
    df = df[df['장소'].isin(min_place)]

    return df

# temp=tmp.copy()

# # 10번 반복합니다.
# for i in range(1,10):
#     temp = preprocessing(temp, 10)
#     print(temp.shape)

# #temp.to_csv('정제.csv', encoding='utf-8')

In [5]:
preprocessing(df,5)

,장소,아이디,평점
1,양재시민의숲,전찬우,5
3,양재시민의숲,김홍일,5
6,양재시민의숲,주니,4
7,양재시민의숲,sang chul kim,5
8,양재시민의숲,jaeho lee,5
...,...,...,...
26649,N서울타워,권홍근,5
26657,N서울타워,Ann,4
26660,N서울타워,김효성,4
26666,N서울타워,김진근,5


In [6]:
df = preprocessing(df,10)
df

,장소,아이디,평점
3,양재시민의숲,김홍일,5
8,양재시민의숲,jaeho lee,5
18,양재시민의숲,METAL actually 메탈액츄얼리 (MEGATDREAM),5
20,양재시민의숲,최광호,5
22,양재시민의숲,장우석,5
...,...,...,...
26590,N서울타워,임승혁,4
26608,N서울타워,김형훈,3
26648,N서울타워,박인영,5
26649,N서울타워,권홍근,5


In [7]:
city_usr_rating = df.pivot_table('평점',index='장소',columns='아이디')
usr_city_rating = df.pivot_table('평점',index='아이디',columns='장소')

In [8]:
city_usr_rating.fillna(0,inplace=True)
city_usr_rating.head()

아이디,--,0 0,0_0 YH,1 1,1004섬 Hwang,1216 ych,1522이성범,2 1,20기김영권,23 1,...,후니,후니아빠,후니훈,후아나,훈,휴휴,흠사마,희야,히어로,히히
장소,,,,,,,,,,,,,,,,,,,,,
168계단,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1913 송정역시장,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0
DDP 동대문디자인플라자,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,4.0,4.0,0.0,4.0,0.0,0.0,4.5,0.0,0.0
N서울타워,0.0,5.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
감천문화마을,0.0,5.0,0.0,4.5,0.0,2.0,0.0,3.0,4.0,0.0,...,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0


In [9]:
similarity_rate = cosine_similarity(city_usr_rating, city_usr_rating)
print(similarity_rate)

[[1.         0.09003981 0.12687065 ... 0.04786223 0.12758643 0.17307596]
 [0.09003981 1.         0.21971775 ... 0.06500155 0.22184344 0.19015872]
 [0.12687065 0.21971775 1.         ... 0.14583675 0.30968363 0.22183152]
 ...
 [0.04786223 0.06500155 0.14583675 ... 1.         0.08136593 0.07226973]
 [0.12758643 0.22184344 0.30968363 ... 0.08136593 1.         0.23856467]
 [0.17307596 0.19015872 0.22183152 ... 0.07226973 0.23856467 1.        ]]


In [10]:
similarity_rate_df = pd.DataFrame(
    data=similarity_rate,
    index = city_usr_rating.index,
    columns = city_usr_rating.index)

In [11]:
similarity_rate_df.head()

장소,168계단,1913 송정역시장,DDP 동대문디자인플라자,N서울타워,감천문화마을,경복궁,경암동철길마을 시작점,경의선숲길,경춘선숲길,광명동굴,...,한라산국립공원,한라수목원,한림공원,해운대해수욕장,행주산성,헤이리 예술마을,홍대쇼핑거리,홍대앞예술시장프리마켓,화담숲,흰여울마을
장소,,,,,,,,,,,,,,,,,,,,,
168계단,1.000000,0.090040,0.126871,0.131534,0.185164,0.127244,0.127518,0.064040,0.052475,0.126020,...,0.079522,0.100611,0.091285,0.043995,0.086526,0.134964,0.085579,0.047862,0.127586,0.173076
1913 송정역시장,0.090040,1.000000,0.219718,0.203613,0.218791,0.192915,0.200433,0.111945,0.106944,0.219770,...,0.115258,0.148147,0.155672,0.052533,0.141677,0.199524,0.133765,0.065002,0.221843,0.190159
DDP 동대문디자인플라자,0.126871,0.219718,1.000000,0.410716,0.330176,0.399926,0.251445,0.190028,0.134213,0.298160,...,0.154468,0.194492,0.210645,0.105768,0.193416,0.278554,0.303817,0.145837,0.309684,0.221832
N서울타워,0.131534,0.203613,0.410716,1.000000,0.337939,0.431580,0.259258,0.156341,0.143856,0.315734,...,0.173418,0.206077,0.209368,0.106736,0.201666,0.282050,0.268892,0.122209,0.321973,0.243993
감천문화마을,0.185164,0.218791,0.330176,0.337939,1.000000,0.325486,0.252819,0.147031,0.123879,0.272405,...,0.164512,0.211224,0.227215,0.107336,0.163988,0.252962,0.229983,0.077214,0.285327,0.303488


In [12]:
def recommend_city(title):
    return similarity_rate_df[title].sort_values(ascending=False)[1:6]

In [17]:
recommend_city('경복궁')

장소
N서울타워            0.431580
DDP 동대문디자인플라자    0.399926
덕수궁              0.396086
광장시장             0.385913
국립중앙박물관          0.384779
Name: 경복궁, dtype: float64